# Práctica 3

**Nombre:** Beatriz Domínguez Urías

**e-mail:** beatriz.dominguez0182@alumnos.udg.mx

## MODULES

In [177]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

## CLASSES

In [2]:
class PlotUtility:
  """Class to simplify creation of plots with plotly graph objects module
  """
  def __init__(self, plot_title: str):
    self.plot_title = plot_title
    self.figure = go.Figure()

  def scatter_plot(self, plot_name: str, x, y, z=None, marker_size=2, line_size=2):
    if z is not None:
      self.figure.add_trace(go.Scatter3d(x = list(x),
                                         y = list(y),
                                         z = list(z),
                                         marker = dict(size=marker_size),
                                         line = dict(width=line_size),
                                         mode = "lines",
                                         name = plot_name,
                                         showlegend = True))
    else:
      self.figure.add_trace(go.Scatter(x = list(x),
                                       y = list(y),
                                       marker = dict(size=marker_size),
                                       line = dict(width=line_size),
                                       mode = "lines",
                                       name = plot_name,
                                       showlegend = True))

  def show_plot(self, xlabel:str=None, ylabel:str=None, zlabel:str=None):
    self.figure.update_layout(title = self.plot_title,
                              scene = dict(xaxis = dict(title= xlabel),
                                           yaxis = dict(title= ylabel),
                                           zaxis = dict(title= zlabel)))
    self.figure.show()
    
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

In [169]:
def bm_2d(n_steps=1000, speed=6, s_pos=[0,0]):
  """
  Arguments:
    n_steps: descripcion1
    speed: descripcion2
    s_pos: descripcion3
  Returns:
    BM_2d_df: decripcion4
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
  temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x, 'y_pos':BM_2d_df.y_pos[i]+velocity.y}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df

def correlated_random_walker(cauchy_exponent, n_steps=1000, speed=6, start_pos=[0,0]):
  """
  Returns a dataframe of a Correlated Random Walk following a cauchy
  distribution of cauchy_exponent with a given value of steps n_steps
  and speed that starts in position start_pos
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  # Init Correlated Random Walk dataframe
  CRW_2d_df = pd.DataFrame(columns = ["x_pos", "y_pos"])
  temp_df = pd.DataFrame([{"x_pos":start_pos[0], "y_pos": start_pos[1]}])
  CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    # Define turn angle following a cauchy distribution
    turn_angle = wrapcauchy.rvs(cauchy_exponent)
    velocity = velocity.rotated(turn_angle)
    # Populate dataframe with walker displacement
    temp_df = pd.DataFrame([{"x_pos":CRW_2d_df.x_pos[i]+velocity.x, "y_pos": CRW_2d_df.y_pos[i]+velocity.y}])
    CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  return CRW_2d_df

def generate_levy_flight(n_steps, speed, miu, beta, levy_exponent, cauchy_exponent, start_pos=[0,0]):
  """
  Returns a dataframe of a Levy Flight following a cauchy distribution for the
  turning angle and a levy stable distribution for the displacement
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  # Init Lévy Flight dataframe
  levy_df = pd.DataFrame(columns = ["x_pos", "y_pos"])
  temp_df = pd.DataFrame([{"x_pos":start_pos[0], "y_pos": start_pos[1]}])
  levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

  # Init steps control variables
  levy_steps = 1
  step_count = 0

  for i in range(n_steps-1):
    if step_count >= levy_steps:
      # Define turn angle following a cauchy distribution
      turn_angle = wrapcauchy.rvs(cauchy_exponent)
      velocity = velocity.rotated(turn_angle)

      # Populate dataframe with walker displacement
      temp_df = pd.DataFrame([{"x_pos":levy_df.x_pos[i]+velocity.x, "y_pos": levy_df.y_pos[i]+velocity.y}])
      levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

      # Reset steps control variables
      levy_steps = levy_stable.rvs(alpha=levy_exponent, beta=beta, loc=miu)
      step_count = 0
    else:
      # Increase a step without a change on direction
      step_count += 1
      temp_df = pd.DataFrame([{"x_pos":levy_df.x_pos[i]+velocity.x, "y_pos": levy_df.y_pos[i]+velocity.y}])
      levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

  return levy_df

def get_euclidean_distance(point_one: pd.Series, point_two: pd.Series) -> float:
  """_summary_

  :param point_one: _description_
  :type point_one: pd.Series
  :param point_two: _description_
  :type point_two: pd.Series
  :return: _description_
  :rtype: float
  """    
  x_comp = math.pow(point_two.iloc[0] - point_one.iloc[0],2)
  y_comp = math.pow(point_two.iloc[1] - point_one.iloc[1],2)
  distance = math.sqrt(x_comp + y_comp)
  
  return distance

def get_path_length(trajectory: pd.DataFrame) -> pd.DataFrame:
  """_summary_

  :param trajectory: _description_
  :type trajectory: pd.DataFrame
  :return: _description_
  :rtype: pd.DataFrame
  """  
  path_length_df = pd.DataFrame(columns = ["distance"])
  for i in range(1, trajectory.shape[0]):
      temp_df = pd.DataFrame([{'distance':get_euclidean_distance(trajectory.iloc[i-1],trajectory.iloc[i])}])
      path_length_df = pd.concat([path_length_df, temp_df], ignore_index=True)

  return path_length_df.cumsum()

def get_mean_squared_displacement(trajectory: pd.DataFrame) -> pd.DataFrame:
  """_summary_

  :param trajectory: _description_
  :type trajectory: pd.DataFrame
  :return: _description_
  :rtype: pd.DataFrame
  """  
  N = trajectory.shape[0]
  MSD_df = pd.DataFrame(columns = ["MSD"])

  for tau in range(1,N):
      displacement_vec = np.array([math.pow(get_euclidean_distance(trajectory.iloc[i-tau], trajectory.iloc[i]), 2)
                              for i in range(tau, BM_2d_df_6.shape[0],1)])
      temp_df = pd.DataFrame([{"MSD": np.mean(displacement_vec)}])
      MSD_df = pd.concat([MSD_df, temp_df], ignore_index=True)
      displacement_vec = np.empty(shape=(0))
  
  return MSD_df

def calculate_vector(pos_a:pd.Series, pos_b:pd.Series) -> list:
    """_summary_

    :param point_a: _description_
    :type point_a: pd.Series
    :param point_b: _description_
    :type point_b: pd.Series
    :return: _description_
    :rtype: list
    """        
    x_comp = pos_b.x_pos - pos_a.x_pos
    y_comp = pos_b.y_pos - pos_a.y_pos
    return [x_comp, y_comp]

###############################################################################################
# Turning angle
# This function calculates the turning angle between three consecutive positions
###############################################################################################
def calculate_turning_angle(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle 
    """
    vec_ab = calculate_vector(pos_a, pos_b)
    norm_ab = np.linalg.norm(vec_ab)
    
    vec_bc = calculate_vector(pos_b, pos_c)
    norm_bc = np.linalg.norm(vec_bc)

    dot_p = np.dot(vec_ab, vec_bc)

    cos_theta = dot_p/(norm_ab*norm_bc+np.finfo(float).eps)

    # Angle orientation
    cross_p = np.cross(vec_ab, vec_bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta

def get_turning_angle_metric(trajectory:pd.DataFrame) -> pd.DataFrame:
    """_summary_

    :param trajectory: _description_
    :type trajectory: pd.DataFrame
    :return: _description_
    :rtype: pd.DataFrame
    """   
    # aux to store turning angles
    aux_ta_traj = np.empty(shape=(0))
    # Iterate over trajectory compute turning angles
    for index, row in trajectory[1:-1].iterrows():
        prev_row = trajectory.iloc[index-1]
        next_row = trajectory.iloc[index+1]
        aux_ta_traj = np.append(aux_ta_traj, calculate_turning_angle(prev_row, row, next_row))

    TA_df = pd.DataFrame(aux_ta_traj, columns= ["TA"])
    return TA_df

def get_step_length_metric(trajectory:pd.DataFrame) -> pd.DataFrame:
  """_summary_

  :param turning_angle_traj: _description_
  :type turning_angle_traj: pd.DataFrame
  :return: _description_
  :rtype: pd.DataFrame
  """    
  TA_traj = get_turning_angle_metric(trajectory)
  aux_sl_traj = np.empty(shape=(0))
  step_length = 0
  for index, row in TA_traj[0:-1].iterrows():
          angle = row.TA
          if angle == 0:
                  step_length += 1
          else:
                  step_length += 1
                  aux_sl_traj = np.append(aux_sl_traj, step_length)
                  step_length = 0
                  
  SL_df = pd.DataFrame(aux_sl_traj, columns= ["SL"])
  return SL_df

## Actividad 1: Path Length - (BM1 vs BM2 vs CRW)

* Implementar función que genere **Brownian Motions** (BM) utilizando **pandas**.
* Implementar función que genere **Correlated Random Walks** (CRW) utilizando pandas.
* Implementar una función alternativa a las ya disponibles en los distintos modulos de python que calcule los valores de la curva de **path length** de una trayectoria.
* Guardar los valores de la métrica en un Data Frame de **pandas**.
* Visualizar con **plotly**.

### (Test) Path Length with reference trajectories

In [ ]:
# Load existing trajectories to test your implementation
# BM speed = 3
BM_2d_df_3 = pd.read_csv('trajectories/brownian_3.csv')

# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [ ]:
# Compute path length (pl) with reference trajectories
## start - Add your code here
ref_pl_BM_3 = get_path_length(BM_2d_df_3)
ref_pl_BM_6 = get_path_length(BM_2d_df_6)
ref_pl_CRW_6_9 = get_path_length(CRW_2d_df_9)
## end - Add your code here

In [ ]:
# Plotting
# Init figure
fig_path_length = PlotUtility("Path length for reference trajectories")
# First trace BM1
## start - Add your code here
fig_path_length.scatter_plot(plot_name = "path length BM 3",
                             x = (ref_pl_BM_3.index)+1,
                             y = ref_pl_BM_3.distance)
## end - Add your code here

# Second trace BM2
## start - Add your code here
fig_path_length.scatter_plot(plot_name = "path length BM 6",
                             x = (ref_pl_BM_6.index)+1,
                             y = ref_pl_BM_6.distance,
                             line_size = 6)    
## end - Add your code here

# Third trace CRW
## start - Add your code here
fig_path_length.scatter_plot(plot_name = "path length CRW 6",
                             x = (ref_pl_CRW_6_9.index)+1,
                             y = ref_pl_CRW_6_9.distance)    
## end - Add your code here

fig_path_length.show_plot()

### Path Length with generated trajectories

In [ ]:
# Generate trajectories
## BM speed = 3
gen_BM_df_3 = bm_2d(speed=3)
## BM speed = 6
gen_BM_df_6 = bm_2d(speed=6)
## CRW speed 6 and cauchy exponent of 0.9
gen_CRW_df_6 = correlated_random_walker(cauchy_exponent=0.9)

# Compute path length (pl) with generated trajectories
gen_pl_BM_3 = get_path_length(gen_BM_df_3)
gen_pl_BM_6 = get_path_length(gen_BM_df_6)
gen_pl_CRW_6 = get_path_length(gen_CRW_df_6)

In [ ]:
# Plotting
# Init figure
fig_gen_pl = PlotUtility("Path length for generated trajectories")
# First trace BM1
## start - Add your code here
fig_gen_pl.scatter_plot(plot_name = "path length BM 3",
                             x = (gen_pl_BM_3.index)+1,
                             y = gen_pl_BM_3.distance)
## end - Add your code here

# Second trace BM2
## start - Add your code here
fig_gen_pl.scatter_plot(plot_name = "path length BM 6",
                             x = (gen_pl_BM_6.index)+1,
                             y = gen_pl_BM_6.distance,
                             line_size = 6)    
## end - Add your code here

# Third trace CRW
## start - Add your code here
fig_gen_pl.scatter_plot(plot_name = "path length CRW 6",
                             x = (gen_pl_CRW_6.index)+1,
                             y = gen_pl_CRW_6.distance)    
## end - Add your code here

fig_gen_pl.show_plot()

## Actividad 2: Mean Squared Displacement - (Brownian vs CRW)

* Generar una trayectoria tipo **BM** y una **CRW**.
* Implementar una función que calcule los valores de la curva de **mean squared displacement** de una trayectoria.
* Guardar metricas en Pandas Data Frame.
* Visualizar con **plotly**.

### (Test) MSD with reference trajectories

In [ ]:
# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [ ]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
                 y = BM_2d_df_6.y_pos,
                 z = BM_2d_df_6.index,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'BM 2d',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
                 y = CRW_2d_df_9.y_pos,
                 z = CRW_2d_df_9.index,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'CRW 2d',
                 showlegend = True))

fig_3d.show()

In [ ]:
# Compute Mean Squared Displacement (MSD) with reference trajectories
## MSD for BM_2d_df_6
MSD_BM_ref = get_mean_squared_displacement(BM_2d_df_6)
## MSD for CRW_2d_df_9
MSD_CRW_ref = get_mean_squared_displacement(CRW_2d_df_9)

In [ ]:
# Init figure
fig_MSD = PlotUtility("Mean Squared Displacement for reference trajectories")

# first trace MSD_BM
## start - Add your code here
fig_MSD.scatter_plot(plot_name = "MSD BM 6",
                     x = MSD_BM_ref.index,
                     y = MSD_BM_ref.MSD)
## end - Add your code here

# Second trace MSD_CRW
## start - Add your code here
fig_MSD.scatter_plot(plot_name = "MSD CRW 6 c=0.9",
                     x = MSD_CRW_ref.index,
                     y = MSD_CRW_ref.MSD)    
## end - Add your code here

fig_MSD.show_plot()

### MSD with generated trajectories

In [ ]:
# Generate trajectories
## BM speed = 6
gen_BM_df_6 = bm_2d(speed=6)
## CRW speed 6 and cauchy exponent of 0.9
gen_CRW_df_6 = correlated_random_walker(cauchy_exponent=0.9)

# Compute Mean Squared Displacement (MSD) with generated trajectories
## MSD for BM of speed 6
MSD_BM = get_mean_squared_displacement(gen_BM_df_6)
## MSD for CRW of speed 6 and cauchy
MSD_CRW = get_mean_squared_displacement(gen_CRW_df_6)

In [ ]:
# Init figure
fig_gen_MSD = PlotUtility("Mean Squared Displacement for generated trajectories")

# first trace MSD_BM
## start - Add your code here
fig_gen_MSD.scatter_plot(plot_name = "MSD BM 6",
                     x = MSD_BM.index,
                     y = MSD_BM.MSD)
## end - Add your code here

# Second trace MSD_CRW
## start - Add your code here
fig_gen_MSD.scatter_plot(plot_name = "MSD CRW 6 c=0.9",
                     x = MSD_CRW.index,
                     y = MSD_CRW.MSD)    
## end - Add your code here

fig_gen_MSD.show_plot()

## Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada)

* Generar **CRWs** con dos exponentes diferentes
* Guardar trayectorias en **Pandas** Data Frame
* Implementar una función que calcule los valores de **turning angle** de una trayectoria.
* Comparar en gráfica distribución origen vs distribución observada (Histograma)
* Visualizar con **plotly**

### (Test) Turning-angle with reference trajectories

In [6]:
# Load existing trajectories to test your implementation
# CRW speed = 6, 
# wrapcauchy [c = 0.6]
CRW_2d_df_6 = pd.read_csv('trajectories/crw_6_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6, 
# wrapcauchy [c = 0.9]
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [7]:
# Compute Turning-angle (TA) with reference trajectories
## TA for CRW_2d_df_6
TA_CRW_6_ref = get_turning_angle_metric(CRW_2d_df_6)
## TA for CRW_2d_df_9
TA_CRW_9_ref = get_turning_angle_metric(CRW_2d_df_9)

In [116]:
# Check documentation
# https://plotly.com/python/histograms/

# PLot histogram
fig_met_df_3 = go.Figure()


# Histogram turning angle CRW_2d_df_6
## start - Add your code here
fig_met_df_3.add_trace(go.Histogram(x=TA_CRW_6_ref.TA, 
                                    histnorm="probability density", 
                                    nbinsx=500, opacity=0.6, 
                                    name="observed_0.6"))   
## end - Add your code here

# Histogram turning angle CRW_2d_df_9
## start - Add your code here
fig_met_df_3.add_trace(go.Histogram(x=TA_CRW_9_ref.TA, 
                                    histnorm="probability density", 
                                    nbinsx=500, 
                                    opacity=0.6, 
                                    name="observed_0.9"))   
## end - Add your code here

# Add origin distributions
## start - Add your code here
resolution = 200
aux_domain_pdf = np.linspace(-np.pi, np.pi, resolution)
cauchy_pdf_6 = np.array([wrapcauchy.pdf(i, 0.6) for i in aux_domain_pdf])
cauchy_pdf_9 = np.array([wrapcauchy.pdf(i, 0.9) for i in aux_domain_pdf])

fig_met_df_3.add_trace(go.Scatter(x = aux_domain_pdf,
                                       y = cauchy_pdf_6,
                                       marker = dict(size=2),
                                       line = dict(width=2, color="blue"),
                                       mode = "lines",
                                       name = "cauchy_0.6",
                                       showlegend = True, 
                                       opacity = 0.6))

fig_met_df_3.add_trace(go.Scatter(x = aux_domain_pdf,
                                       y = cauchy_pdf_9,
                                       marker = dict(size=2),
                                       line = dict(width=2, color="red"),
                                       mode = "lines",
                                       name = "cauchy_0.9",
                                       showlegend = True,
                                       opacity = 0.6))
## end - Add your code here

fig_met_df_3.show()

### Turning-angle with generated trajectories

In [117]:
# Compute Turning-angle (TA) with generated trajectories
## TA for CRW_2d_df_6
CRW_6_df = correlated_random_walker(0.6)
TA_CRW_6_gen = get_turning_angle_metric(CRW_6_df)
## TA for CRW_2d_df_9
CRW_9_df = correlated_random_walker(0.9)
TA_CRW_9_gen = get_turning_angle_metric(CRW_9_df)

In [118]:
# Check documentation
# https://plotly.com/python/histograms/

# PLot histogram
fig_met_gen_3 = go.Figure()


# Histogram turning angle CRW_2d_df_6
## start - Add your code here
fig_met_gen_3.add_trace(go.Histogram(x=TA_CRW_6_gen.TA, 
                                    histnorm="probability density", 
                                    nbinsx=500, opacity=0.6, 
                                    name="observed_0.6"))   
## end - Add your code here

# Histogram turning angle CRW_2d_df_9
## start - Add your code here
fig_met_gen_3.add_trace(go.Histogram(x=TA_CRW_9_gen.TA, 
                                    histnorm="probability density", 
                                    nbinsx=500, 
                                    opacity=0.6, 
                                    name="observed_0.9"))   
## end - Add your code here

# Add origin distributions
## start - Add your code here
resolution = 200
aux_domain_pdf = np.linspace(-np.pi, np.pi, resolution)
cauchy_pdf_6 = np.array([wrapcauchy.pdf(i, 0.6) for i in aux_domain_pdf])
cauchy_pdf_9 = np.array([wrapcauchy.pdf(i, 0.9) for i in aux_domain_pdf])

fig_met_gen_3.add_trace(go.Scatter(x = aux_domain_pdf,
                                       y = cauchy_pdf_6,
                                       marker = dict(size=2),
                                       line = dict(width=2, color="blue"),
                                       mode = "lines",
                                       name = "cauchy_0.6",
                                       showlegend = True, 
                                       opacity = 0.6))

fig_met_gen_3.add_trace(go.Scatter(x = aux_domain_pdf,
                                       y = cauchy_pdf_9,
                                       marker = dict(size=2),
                                       line = dict(width=2, color="red"),
                                       mode = "lines",
                                       name = "cauchy_0.9",
                                       showlegend = True,
                                       opacity = 0.6))
## end - Add your code here

fig_met_gen_3.show()

## Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada)

* Implementar función que genere **Lévy Walks** (LW) utilizando pandas.
* Guardar trayectorias en Pandas Data Frame.
* Implementar una función que calcule los valores de **step lenght** de una trayectoria.
* Guardar trayectorias en **pandas** Data Frame.
* Obtener **Step-length** distribution.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con plotly.

In [120]:
# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=1.0, beta=1.0, loc=3.0]
Levy_2d_df_1 = pd.read_csv('trajectories/levy_6_1.csv')

# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=0.7, beta=1.0, loc=3.0]
Levy_2d_df_7 = pd.read_csv('trajectories/levy_6_7.csv')

In [170]:
# Compute Step-Length (SL) with reference trajectories
SL_LF_1 = get_step_length_metric(Levy_2d_df_1)
SL_LF_7 = get_step_length_metric(Levy_2d_df_7)
      

In [181]:
# PLot histogram
fig_met_df_4 = go.Figure()

# Histogram step-length Levy_2d_df_1
## start - Add your code here
fig_met_df_4.add_trace(go.Histogram(x=SL_LF_1.SL, 
                                    histnorm="probability density", 
                                    xbins=dict(start=0, end=50, size=0.5), 
                                    name="Observed_alpha=1.0_beta=1.0",
                                    opacity=0.6))
## end - Add your code here


# Histogram step-length Levy_2d_df_7
## start - Add your code here
fig_met_df_4.add_trace(go.Histogram(x=SL_LF_7.SL, 
                                    histnorm="probability density", 
                                    xbins=dict(start=0, end=50, size=0.5), 
                                    name="Observed_alpha=0.7_beta=1.0",
                                    opacity=0.6))    
## end - Add your code here


# Add origin distributions
## start - Add your code here
resolution = 200
aux_domain_levy = np.linspace(0, 50, resolution)
levy_pdf_1 = np.array([levy_stable.pdf(i, alpha=1.0, beta=1.0, loc=5.0) for i in aux_domain_levy])
levy_pdf_7 = np.array([levy_stable.pdf(i, alpha=0.7, beta=1.0, loc=5.0) for i in aux_domain_levy])

fig_met_df_4.add_trace(go.Scatter(x = aux_domain_levy,
                                       y = levy_pdf_1,
                                       marker = dict(size=2),
                                       line = dict(width=2, color="blue"),
                                       mode = "lines",
                                       name = "Levy_alpha=1.0",
                                       showlegend = True, 
                                       opacity = 0.6))

fig_met_df_4.add_trace(go.Scatter(x = aux_domain_levy,
                                       y = levy_pdf_7,
                                       marker = dict(size=2),
                                       line = dict(width=2, color="red"),
                                       mode = "lines",
                                       name = "Levy_alpha=0.7",
                                       showlegend = True,
                                       opacity = 0.6))    
## end - Add your code here


fig_met_df_4.show()